## The System:

![grafik.png][def]


[def]: .\PTG_System.png

## Preperation

In [ ]:
#!pip install gurobipy==10.0.0
#!pip install pyomo==6.6.2
#!pip install numpy==1.24.3
#!pip install pandas==1.3.5
#!pip install matplotlib

In [ ]:
from gurobipy import *
from pyomo.environ import *
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tsam.timeseriesaggregation as tsam


## Time Agreegation

load time series data


In [ ]:
# Time series data of a whole year with hourly resolution
time_series_data = pd.read_excel(".\Data_Sheet_neu_2.xlsx")


print(time_series_data)

In [ ]:
# Erstelle eine Figur mit 3 nebeneinanderliegenden Subplots
fig, axes = plt.subplots(1, 4, figsize=(18, 5))

# Plot für den ersten Datensatz
axes[0].plot(time_series_data['PowerD_demand'], label='el_demand_rel')
axes[0].set_xlabel('Time step')
axes[0].set_ylabel('Relative value')
axes[0].legend()
axes[0].set_title('Electricity Demand')

# Plot für den zweiten Datensatz
axes[1].plot(time_series_data['HeatD_demand'], label='h_demand_rel')
axes[1].set_xlabel('Time step')
axes[1].set_ylabel('Relative value')
axes[1].legend()
axes[1].set_title('Heat Demand')

# Plot für den dritten Datensatz
axes[2].plot(time_series_data['P_PV_tot'], label='P_Pv_rel')
axes[2].set_xlabel('Time step')
axes[2].set_ylabel('Relative value')
axes[2].legend()
axes[2].set_title('PV Power')

# Plot für den dritten Datensatz
axes[3].plot(time_series_data['electricity_price'], label='c_el')
axes[3].set_xlabel('Time step')
axes[3].set_ylabel('Relative value')
axes[3].legend()
axes[3].set_title('Electricity Price c_el in €/kwh')

# Layout anpassen
plt.tight_layout()

# Zeige die Plots
plt.show()

Time series aggregation


In [ ]:
# aggregate time_series_data with the tsam package
noTypicalPeriods = 10
hoursPerPeriod = 24

aggregation = tsam.TimeSeriesAggregation(
        timeSeries                  = time_series_data,
        noTypicalPeriods            = noTypicalPeriods,
        hoursPerPeriod              = hoursPerPeriod,
        clusterMethod               = 'k_means',
        resolution                  = 1
    )
typical_day_time_series_data = aggregation.createTypicalPeriods()

print(typical_day_time_series_data)

In [ ]:
# Assign labels to each original period            #NEW
typical_days_row = aggregation.clusterOrder
print("Cluster Labels:")
print(typical_days_row)
print(len(typical_days_row))

In [ ]:
# print the number of days represented by each typical day
for typical_day in np.arange(noTypicalPeriods):
    print(f"Typical day {typical_day} represents {aggregation.clusterPeriodNoOccur[typical_day]} days in the original time series")
    

Für Seasonal_Storrage Zeitreihen erstellen

In [ ]:
days_yeahr=[i for i in range(0, 365)]  #NEW
print(days_yeahr)

In [ ]:
typical_days_dict = dict(zip(days_yeahr, typical_days_row))   #NEW
print(typical_days_dict)

In [ ]:
typical_hours_row =[] #

    
for i in range(0, 365):  #NEW
    #print(i)
    for j in range(24):
        typical_hours_row.append(typical_days_dict[i]*24+j)
        
print(len(typical_hours_row))
print(typical_hours_row)

In [ ]:
# Erstellen der Zahlenreihe von 0 bis 8759
zahlenreihe = list(range(0, 8760))

In [ ]:
hours_to_typical_days= dict(zip(zahlenreihe,typical_hours_row))  #NEW
print(hours_to_typical_days)

In [ ]:
# reindex time series to have unique single-dimensional index starting from 0
typical_day_time_series_data = typical_day_time_series_data.reset_index()
typical_day_time_series_data = typical_day_time_series_data.rename(columns={'level_0': 'typical_day'})
typical_day_time_series_data.index = np.arange(0, noTypicalPeriods * hoursPerPeriod)
#TimeStep von 1 bis 95 laufen lassen:
typical_day_time_series_data['TimeStep'] = np.arange(0, noTypicalPeriods * hoursPerPeriod)
print(typical_day_time_series_data)

In [ ]:
#modified_time_series_data.to_excel('modified_time_series_data_new.xlsx', index=False)


## Time series
#### Daten laden

In [ ]:
print(time_series_data)

In [ ]:
print(typical_day_time_series_data)

In [ ]:
time_slices = np.array(typical_day_time_series_data.index)

shifted_time_slices = [] # time slice number shifted by 1 to create the storage level constraint
time_slice_weight = []

for typical_period in np.arange(noTypicalPeriods):
    time_slices_within_typical_period = typical_day_time_series_data[typical_day_time_series_data.typical_day == typical_period].index
    shifted_time_slices_within_period = np.roll(time_slices_within_typical_period,-1)
    shifted_time_slices.extend(shifted_time_slices_within_period) 
    time_slice_weight.extend(aggregation.clusterPeriodNoOccur[typical_period]*np.ones(len(time_slices_within_typical_period)))
    
print(shifted_time_slices)
print(time_slice_weight)

# weight of each time slice [h]
time_slice_weight = dict(zip(time_slices, time_slice_weight))

# consecutive time slice of each time slice
next_time_slice = dict(zip(time_slices, shifted_time_slices))

In [ ]:
# Erstellen des Dictionaries
storage_next_time_slices = {zahlenreihe[i]: zahlenreihe[(i + 1) % len(zahlenreihe)] for i in range(len(zahlenreihe))}

# Ausgabe des Ergebnisses
print(storage_next_time_slices)

In [ ]:
print(time_slices)

In [ ]:
 # gas price of each time slice
power_demand_time_series = dict(zip(time_slices, np.array(typical_day_time_series_data['PowerD_demand'])))

# # heat demand of each time slice
heat_demand_time_series = dict(zip(time_slices, np.array(typical_day_time_series_data['HeatD_demand'])))

# # electricity_price of each time slice
electricity_price_time_series = dict(zip(time_slices, np.array(typical_day_time_series_data['electricity_price'])))

# # PV_Power of eacj time slice
power_PV_time_series = dict(zip(time_slices, np.array(typical_day_time_series_data['P_PV_tot'])))
for timeslice in power_PV_time_series:
        if power_PV_time_series[timeslice] > 900:
            power_PV_time_series[timeslice] = 900


## Setting up the model:

In [ ]:
model = ConcreteModel()


# declare sets
model.time_slices = set(time_slices)
model.time_slices_through_whole_yeahr = set(zahlenreihe)

In [ ]:
print(model.time_slices)
print(model.time_slices_through_whole_yeahr)

In [ ]:
# declare decision variables
model.OPEX = Var(domain=NonNegativeReals) # [€]

# declare time dependent decision variables
model.P_e_buy = Var(model.time_slices, domain=NonNegativeReals) # Power, that needs to get bought from the grid [kW_e]
model.P_CH4_sell = Var(model.time_slices, domain=NonNegativeReals) # CH4, that gets to sell to the grid [kW_CH4]
model.mdot_CO2_atm = Var(model.time_slices, domain=NonNegativeReals) # CO2 emissions [kg_CO2/hr]

# Component_Inputs
model.P_e_PEM = Var(model.time_slices, domain=NonNegativeReals) # Electrolysis [kW_e]
model.P_e_DAC = Var(model.time_slices, domain=NonNegativeReals) # DirectAirCapture [kW_e]
model.P_H2_MR = Var(model.time_slices, domain=NonNegativeReals) # MethaneReformation [kW_H2]
model.mdot_CO2_MR = Var(model.time_slices, domain=NonNegativeReals) # MethaneReformation [kW_CO2] STIMMT EINHEIT???
model.P_CH4_CHP = Var(model.time_slices, domain=NonNegativeReals) # CombinedHeatPower [kW_CH4]
model.P_CH4_BO = Var(model.time_slices, domain=NonNegativeReals) # Boiler [kW_CH4]

model.y_PEM = Var(model.time_slices, domain=Binary) # Electrolysis, on/off
model.y_CHP = Var(model.time_slices, domain=Binary) # CombinedHeatPower, on/off
model.y_BO = Var(model.time_slices, domain=Binary) # Boiler, on/off

# Component_Outputs
model.P_e_PV = Var(model.time_slices, domain=NonNegativeReals) # PV [kW_e]
model.P_H2_PEM = Var(model.time_slices, domain=NonNegativeReals) # Electrolysis [kW_H2]
model.mdot_CO2_DAC = Var(model.time_slices, domain=NonNegativeReals) # DirectAirCapture [kg_CO2/hr]
model.P_CH4_MR = Var(model.time_slices, domain=NonNegativeReals) # MethaneReformation [kW_CH4]
model.P_e_CHP = Var(model.time_slices, domain=NonNegativeReals) # CombinedHeatPower [kW_e]
model.P_h_CHP = Var(model.time_slices, domain=NonNegativeReals) # CombinedHeatPower [kW_h]
model.P_h_BO = Var(model.time_slices, domain=NonNegativeReals) # Boiler [kW_h]                      

# Storages
model.P_e_battery = Var(model.time_slices, domain=Reals) # [negative: storage charge, positive: storage discharge] [kW_e]  #hier ist die Frage_model.time_slices oder model.time_slices_through_whole_yeahr
model.P_e_battery_in = Var(model.time_slices_through_whole_yeahr, domain=NonNegativeReals) # [kW_e] 
model.P_e_battery_out = Var(model.time_slices_through_whole_yeahr, domain=NonNegativeReals) #  [kW_e]
model.storage_level_battery = Var(model.time_slices_through_whole_yeahr, domain=NonNegativeReals) # [kWh]
model.in_battery =  Var(model.time_slices_through_whole_yeahr, domain=Binary) #einspeichern
model.out_battery = Var(model.time_slices_through_whole_yeahr, domain=Binary) #ausspeichern
model.P_e_battery_yearly = Var(model.time_slices_through_whole_yeahr, domain=Reals) # [kW_e]

model.P_h_thermalStorage = Var(model.time_slices, domain=Reals) # [negative: storage charge, positive: storage discharge] [kW_h]
model.P_h_thermalStorage_in = Var(model.time_slices_through_whole_yeahr, domain=NonNegativeReals) # [kW_h]
model.P_h_thermalStorage_out = Var(model.time_slices_through_whole_yeahr, domain=NonNegativeReals) # [kW_h]
model.storage_level_thermalStorage = Var(model.time_slices_through_whole_yeahr, domain=NonNegativeReals) # [kWh]
model.in_thermalStorage =  Var(model.time_slices_through_whole_yeahr, domain=Binary) #einspeichern
model.out_thermalStorage = Var(model.time_slices_through_whole_yeahr, domain=Binary) #ausspeichern
model.P_h_thermalStorage_yearly = Var(model.time_slices_through_whole_yeahr,domain=Reals) # [kW_h]

model.P_H2_gasStorage = Var(model.time_slices, domain=Reals) # [negative: storage charge, positive: storage discharge] [kW_H2]
model.P_H2_gasStorage_in = Var(model.time_slices_through_whole_yeahr, domain=NonNegativeReals) # [kW_H2]
model.P_H2_gasStorage_out = Var(model.time_slices_through_whole_yeahr, domain=NonNegativeReals) # [kW_H2]
model.storage_level_H2Storage = Var(model.time_slices_through_whole_yeahr, domain=NonNegativeReals) # [kWh_H2]
model.in_H2Storage =  Var(model.time_slices_through_whole_yeahr, domain=Binary) #einspeichern
model.out_H2Storage = Var(model.time_slices_through_whole_yeahr, domain=Binary) #ausspeichern
model.P_H2_gasStorage_yearly = Var(model.time_slices_through_whole_yeahr,domain=Reals) # [kW_H2]

model.mdot_CO2_gasStorage = Var(model.time_slices, domain=Reals) # [negative: storage charge, positive: storage discharge] [kg_CO2/hr]
model.mdot_CO2_gasStorage_in = Var(model.time_slices_through_whole_yeahr, domain=NonNegativeReals) # [kg_CO2/hr]
model.mdot_CO2_gasStorage_out = Var(model.time_slices_through_whole_yeahr, domain=NonNegativeReals) # [kg_CO2/hr]
model.storage_level_CO2Storage = Var(model.time_slices_through_whole_yeahr, domain=NonNegativeReals) # [kg_CO2]
model.in_CO2Storage =  Var(model.time_slices_through_whole_yeahr, domain=Binary) #einspeichern
model.out_CO2Storage = Var(model.time_slices_through_whole_yeahr, domain=Binary) #ausspeichern
model.mdot_CO2_gasStorage_yearly = Var(model.time_slices_through_whole_yeahr,domain=Reals) # [kg_CO2/hr]

model.P_CH4_gasStorage = Var(model.time_slices, domain=Reals) # [negative: storage charge, positive: storage discharge] [kW_CH4]
model.P_CH4_gasStorage_in = Var(model.time_slices_through_whole_yeahr, domain=NonNegativeReals) # [kW_CH4]
model.P_CH4_gasStorage_out = Var(model.time_slices_through_whole_yeahr, domain=NonNegativeReals) # [kW_CH4]
model.storage_level_CH4Storage = Var(model.time_slices_through_whole_yeahr, domain=NonNegativeReals) # [kWh_CH4]
model.in_CH4Storage =  Var(model.time_slices_through_whole_yeahr, domain=Binary) #einspeichern
model.out_CH4Storage = Var(model.time_slices_through_whole_yeahr, domain=Binary) #ausspeichern
model.P_CH4_gasStorage_yearly = Var(model.time_slices_through_whole_yeahr,domain=Reals) # [kW_CH4]

In [ ]:
# declare parameters

model.price_co2 = 0.0844 # [€/ton]
model.price_gas = 0.0775 # [€/kWh]

model.co2_ch4_relation = 2.744 #[kg_CO2/kg_CH4]

model.P_e_min_PEM = 180 #[kW_e]
model.P_e_max_PEM = 1080 #[kW_e]

model.mdot_CO2_N_DAC = 250 #[kg_CO2/hr]
model.eta_DAC = 0.5 #[kg_CO2/kWh_e]

model.P_CH4_N_MR = 1500 #[kW_CH4]
model.eta_MR = 0.78
model.co2need_MR = 0.178 #E_in_CO2/E_out_CH4 [kg_CO2/kWh_CH4]

model.P_h_min_CHP = 400 #[kW_h]
model.P_h_max_CHP = 800 #[kW_h]

model.P_h_min_BO = 0.5 #[kW_h]
model.P_h_max_BO = 500 #[kW_h]

model.E_N_battery = 5000 #[kWh_e]
model.eta_battery = 0.92 
model.loss_battery = 0.000042 #[1/hr]
model.c_battery = 0.36 #[1/hr]

model.Q_N_thermalStorage = 5000 #[kWh_Q]
model.eta_thermalStorage = 0.95 
model.loss_thermalStorage = 0.005 #[1/hr]
model.c_thermalStorage = 1 #[1/hr]

model.E_N_H2Storage = 5000 #[kWh_H2]
model.m_N_CO2Storage = 5000 #[kg_CO2]
model.E_N_CH4Storage = 5000 #[kWh_CH4]
model.loss_gasTanks = 0 #[1/hr]
model.eta_gasTanks = 1 
model.c_gasTanks = 0.25 #[1/hr]


In [ ]:
#declare time dependent parameters
model.P_e_demand = Param(model.time_slices, initialize = power_demand_time_series)
model.Q_demand = Param(model.time_slices, initialize = heat_demand_time_series)

model.electricity_price = Param(model.time_slices, initialize = electricity_price_time_series)

model.P_PV = Param(model.time_slices, initialize = power_PV_time_series)


In [ ]:
# declare unindexed constraints
model.OPEX_constraint = Constraint(expr =  model.OPEX == sum(model.electricity_price[time_slice]*model.P_e_buy[time_slice] for time_slice in model.time_slices) 
                                     -  sum(model.price_gas * model.P_CH4_sell[time_slice] for time_slice in model.time_slices)
                                    +  sum(model.price_co2 * model.mdot_CO2_atm[time_slice] for time_slice in model.time_slices))

# declare rules to setup indexed constraints
def electricty_balance_constraint_rule(model, time_slice):
    return (model.P_e_PV[time_slice] + model.P_e_CHP[time_slice] + model.P_e_battery[time_slice] + model.P_e_buy[time_slice] == model.P_e_demand[time_slice] + model.P_e_PEM[time_slice] + model.P_e_DAC[time_slice])

def heat_balance_constraint_rule(model, time_slice):
    return (model.P_h_CHP[time_slice] + model.P_h_BO[time_slice] + model.P_h_thermalStorage[time_slice] == model.Q_demand[time_slice])

def h2_balance_constraint_rule(model, time_slice):
    return (model.P_H2_PEM[time_slice] + model.P_H2_gasStorage[time_slice] == model.P_H2_MR[time_slice])

def co2_balance_contraint_rule(model, time_slice):
    return (model.mdot_CO2_DAC[time_slice] + model.mdot_CO2_gasStorage[time_slice] == model.mdot_CO2_MR[time_slice])

def ch4_balance_constraint_rule(model, time_slice):
    return (model.P_CH4_MR[time_slice] + model.P_CH4_gasStorage[time_slice] == model.P_CH4_CHP[time_slice] + model.P_CH4_BO[time_slice] + model.P_CH4_sell[time_slice])

def co2_emmission_constraint_rule(model, time_slice):
    return (model.mdot_CO2_atm[time_slice] == model.co2_ch4_relation * (model.P_CH4_BO[time_slice] + model.P_CH4_CHP[time_slice]))



def PV_constraint_rule(model, time_slice):
    return (model.P_e_PV[time_slice] == model.P_PV[time_slice])

def PEM_constraint_rule(model, time_slice):
    return (model.P_H2_PEM[time_slice] == 47.877 * model.y_PEM[time_slice] + 0.549 * model.P_e_PEM[time_slice])
def PEM_minInput_constraint_rule(model, time_slice):
    return (model.P_e_PEM[time_slice] >= model.P_e_min_PEM * model.y_PEM[time_slice])
def PEM_maxInput_constraint_rule(model, time_slice):
    return (model.P_e_PEM[time_slice] <= model.P_e_max_PEM* model.y_PEM[time_slice])

def DAC_constraint_rule(model, time_slice):
    return (model.mdot_CO2_DAC[time_slice] == model.eta_DAC * model.P_e_DAC[time_slice])
def DAC_maxOuput_constraint_rule(model, time_slice):
    return (model.mdot_CO2_DAC[time_slice] <= model.mdot_CO2_N_DAC)

def MR_constraint_rule(model, time_slice):
    return (model.P_CH4_MR[time_slice] == model.eta_MR * model.P_H2_MR[time_slice])
def MR_CO2_constraint_rule(model, time_slice):
    return (model.mdot_CO2_MR[time_slice] == model.co2need_MR * model.P_CH4_MR[time_slice])
def MR_maxOuput_constraint_rule(model, time_slice):
    return (model.P_CH4_MR[time_slice] <= model.P_CH4_N_MR)

def CHP_heat_constraint_rule(model, time_slice):
    return (model.P_h_CHP[time_slice] == 0.47 * model.P_CH4_CHP[time_slice])

def CHP_heat_constraint_rule(model, time_slice):
    return model.P_h_CHP[time_slice] == -2416.78 * model.y_CHP[time_slice] + 1.90 * model.P_CH4_CHP[time_slice] 
def CHP_heat_minOuput_constraint_rule(model, time_slice):
    return model.P_h_CHP[time_slice] >= model.P_h_min_CHP * model.y_CHP[time_slice]
def CHP_heat_maxOuput_constraint_rule(model, time_slice):
    return model.P_h_CHP[time_slice] <= model.P_h_max_CHP * model.y_CHP[time_slice]
def CHP_electricity_constraint_rule(model, time_slice):
    return (model.P_e_CHP[time_slice] == -2042.74 * model.y_CHP[time_slice] + 1.61 * model.P_CH4_CHP[time_slice]) 
    
def BO_constraint_rule(model, time_slice):
    return (model.P_CH4_BO[time_slice]  == 113.95 * model.y_BO[time_slice] + 0.82 * model.P_h_BO[time_slice])
def BO_minOuput_constraint_rule(model, time_slice):
    return (model.P_h_BO[time_slice] >= model.P_h_min_BO * model.y_BO[time_slice])
def BO_maxOuput_constraint_rule(model, time_slice):
    return (model.P_h_BO[time_slice] <= model.P_h_max_BO * model.y_BO[time_slice])



#Battery
def battery_level_constraint_rule(model, time_slice_through_whole_yeahr):
    return (model.storage_level_battery[storage_next_time_slices[time_slice_through_whole_yeahr]] == 
        model.storage_level_battery[time_slice_through_whole_yeahr] * (1-model.loss_battery)
        + (model.P_e_battery_in[time_slice_through_whole_yeahr]*model.eta_battery - model.P_e_battery_out[time_slice_through_whole_yeahr]/model.eta_battery))
    
def battery_Energy_balance_constraint_rule(model, time_slice_through_whole_yeahr):
    return  (model.P_e_battery_yearly[time_slice_through_whole_yeahr] == -model.P_e_battery_in[time_slice_through_whole_yeahr] + model.P_e_battery_out[time_slice_through_whole_yeahr])
    
def battery_level_max_in_constraint_rule(model, time_slice_through_whole_yeahr):
    return  (model.P_e_battery_in[time_slice_through_whole_yeahr] <= model.in_battery[time_slice_through_whole_yeahr]*model.c_battery*model.E_N_battery)
    
def battery_level_max_out_constraint_rule(model, time_slice_through_whole_yeahr):
    return  (model.P_e_battery_out[time_slice_through_whole_yeahr] <= model.out_battery[time_slice_through_whole_yeahr]*model.c_battery*model.E_N_battery)

def battery_in_or_out_or_off_constraint_rule(model, time_slice_through_whole_yeahr):
    return  (model.in_battery[time_slice_through_whole_yeahr] + model.out_battery[time_slice_through_whole_yeahr] <= 1)

def connect_battery_time_slices_rule(model, time_slices_though_whole_year):  # NEW---??
    return model.P_e_battery_yearly[time_slices_though_whole_year] == model.P_e_battery[hours_to_typical_days[time_slices_though_whole_year]]



#thermalStorage
def thermalStorage_level_constraint_rule(model, time_slice_through_whole_yeahr):
    return  (model.storage_level_thermalStorage[storage_next_time_slices[time_slice_through_whole_yeahr]] == (
        model.storage_level_thermalStorage[time_slice_through_whole_yeahr] * (1-model.loss_thermalStorage)
        + (model.P_h_thermalStorage_in[time_slice_through_whole_yeahr]*model.eta_thermalStorage - model.P_h_thermalStorage_out[time_slice_through_whole_yeahr]/model.eta_thermalStorage)))

def thermalStorage_Energy_balance_constraint_rule(model, time_slices_through_whole_yeahr):  
    return  (model.P_h_thermalStorage_yearly[time_slices_through_whole_yeahr] == - model.P_h_thermalStorage_in[time_slices_through_whole_yeahr] + model.P_h_thermalStorage_out[time_slices_through_whole_yeahr])  

def thermalStorage_level_max_in_constraint_rule(model, time_slices_through_whole_yeahr):        
    return  (model.P_h_thermalStorage_in[time_slices_through_whole_yeahr] <= model.in_thermalStorage[time_slices_through_whole_yeahr]*model.c_thermalStorage*model.Q_N_thermalStorage)

def thermalStorage_level_max_out_constraint_rule(model, time_slices_through_whole_yeahr):    
    return  (model.P_h_thermalStorage_out[time_slices_through_whole_yeahr] <= model.out_thermalStorage[time_slices_through_whole_yeahr]*model.c_thermalStorage*model.Q_N_thermalStorage)

def thermalStorage_in_or_out_or_off_constraint_rule(model, time_slices_through_whole_yeahr):    
    return  (model.in_thermalStorage[time_slices_through_whole_yeahr] + model.out_thermalStorage[time_slices_through_whole_yeahr] <= 1)

def connect_thermalStorage_time_slices_rule(model, time_slices_though_whole_year):  # NEW---??
    return model.P_h_thermalStorage_yearly[time_slices_though_whole_year] == model.P_h_thermalStorage[hours_to_typical_days[time_slices_though_whole_year]]


#H2Storage
def H2Storage_level_constraint_rule(model, time_slices_through_whole_yeahr):
    return  (model.storage_level_H2Storage[storage_next_time_slices[time_slices_through_whole_yeahr]] == (
        model.storage_level_H2Storage[time_slices_through_whole_yeahr] * (1-model.loss_gasTanks)
        + (model.P_H2_gasStorage_in[time_slices_through_whole_yeahr]*model.eta_gasTanks - model.P_H2_gasStorage_out[time_slices_through_whole_yeahr]/model.eta_gasTanks)))   
    
def H2Storage_Energy_balance_constraint_rule(model, time_slices_through_whole_yeahr):
    return  (model.P_H2_gasStorage_yearly[time_slices_through_whole_yeahr] == -model.P_H2_gasStorage_in[time_slices_through_whole_yeahr] + model.P_H2_gasStorage_out[time_slices_through_whole_yeahr]) 

def H2Storage_level_max_in_constraint_rule(model, time_slices_through_whole_yeahr):
    return  (model.P_H2_gasStorage_in[time_slices_through_whole_yeahr] <= model.in_H2Storage[time_slices_through_whole_yeahr]*model.c_gasTanks*model.E_N_H2Storage)

def H2Storage_level_max_out_constraint_rule(model, time_slices_through_whole_yeahr):
    return  (model.P_H2_gasStorage_out[time_slices_through_whole_yeahr] <= model.out_H2Storage[time_slices_through_whole_yeahr]*model.c_gasTanks*model.E_N_H2Storage)

def H2Storage_in_or_out_or_off_constraint_rule(model, time_slices_through_whole_yeahr):
    return  (model.in_H2Storage[time_slices_through_whole_yeahr] + model.out_H2Storage[time_slices_through_whole_yeahr] <= 1)

def connect_H2Storage_time_slices_rule(model, time_slices_though_whole_year):  # NEW---??
    return model.P_H2_gasStorage_yearly[time_slices_though_whole_year] == model.P_H2_gasStorage[hours_to_typical_days[time_slices_though_whole_year]]


#CO2Storage
def CO2Storage_level_constraint_rule(model, time_slices_through_whole_yeahr):    
    return  (model.storage_level_CO2Storage[storage_next_time_slices[time_slices_through_whole_yeahr]] == (
        model.storage_level_CO2Storage[time_slices_through_whole_yeahr] * (1-model.loss_gasTanks)
        + (model.mdot_CO2_gasStorage_in[time_slices_through_whole_yeahr]*model.eta_gasTanks - model.mdot_CO2_gasStorage_out[time_slices_through_whole_yeahr]/model.eta_gasTanks)))   
    
def CO2Storage_Energy_balance_constraint_rule(model, time_slices_through_whole_yeahr):
    return  (model.mdot_CO2_gasStorage_yearly[time_slices_through_whole_yeahr] == -model.mdot_CO2_gasStorage_in[time_slices_through_whole_yeahr] + model.mdot_CO2_gasStorage_out[time_slices_through_whole_yeahr])  

def CO2Storage_level_max_in_constraint_rule(model, time_slices_through_whole_yeahr):
    return  (model.mdot_CO2_gasStorage_in[time_slices_through_whole_yeahr] <= model.in_CO2Storage[time_slices_through_whole_yeahr]*model.c_gasTanks*model.m_N_CO2Storage)

def CO2Storage_level_max_out_constraint_rule(model, time_slices_through_whole_yeahr):
    return  (model.mdot_CO2_gasStorage_out[time_slices_through_whole_yeahr] <= model.out_CO2Storage[time_slices_through_whole_yeahr]*model.c_gasTanks*model.m_N_CO2Storage)

def CO2Storage_in_or_out_or_off_constraint_rule(model, time_slices_through_whole_yeahr):
    return  (model.in_CO2Storage[time_slices_through_whole_yeahr] + model.out_CO2Storage[time_slices_through_whole_yeahr] <= 1)

def connect_CO2Storage_time_slices_rule(model, time_slices_though_whole_year):  # NEW---??
    return model.mdot_CO2_gasStorage_yearly[time_slices_though_whole_year] == model.mdot_CO2_gasStorage[hours_to_typical_days[time_slices_though_whole_year]]


#CH4Storage	
def CH4Storage_level_constraint_rule(model, time_slices_through_whole_yeahr):
    return  (model.storage_level_CH4Storage[storage_next_time_slices[time_slices_through_whole_yeahr]] == (
        model.storage_level_CH4Storage[time_slices_through_whole_yeahr] * (1-model.loss_gasTanks)
        + (model.P_CH4_gasStorage_in[time_slices_through_whole_yeahr]*model.eta_gasTanks - model.P_CH4_gasStorage_out[time_slices_through_whole_yeahr]/model.eta_gasTanks)))

def CH4Storage_Energy_balance_constraint_rule(model, time_slices_through_whole_yeahr):
    return  (model.P_CH4_gasStorage_yearly[time_slices_through_whole_yeahr] == -model.P_CH4_gasStorage_in[time_slices_through_whole_yeahr] + model.P_CH4_gasStorage_out[time_slices_through_whole_yeahr])

def CH4Storage_level_max_in_constraint_rule(model, time_slices_through_whole_yeahr):
    return  (model.P_CH4_gasStorage_in[time_slices_through_whole_yeahr] <= model.in_CH4Storage[time_slices_through_whole_yeahr]*model.c_gasTanks*model.E_N_CH4Storage)

def CH4Storage_level_max_out_constraint_rule(model, time_slices_through_whole_yeahr):
    return  (model.P_CH4_gasStorage_out[time_slices_through_whole_yeahr] <= model.out_CH4Storage[time_slices_through_whole_yeahr]*model.c_gasTanks*model.E_N_CH4Storage)

def CH4Storage_in_or_out_or_off_constraint_rule(model, time_slices_through_whole_yeahr):
    return  (model.in_CH4Storage[time_slices_through_whole_yeahr] + model.out_CH4Storage[time_slices_through_whole_yeahr] <= 1)

def connect_CH4Storage_time_slices_rule(model, time_slices_though_whole_year):  # NEW---??
    return model.P_CH4_gasStorage_yearly[time_slices_though_whole_year] == model.P_CH4_gasStorage[hours_to_typical_days[time_slices_though_whole_year]]


# declare indexed constraints

model.CH4Storage_level_constraint = Constraint(model.time_slices_through_whole_yeahr, rule=CH4Storage_level_constraint_rule)
model.CH4Storage_Energy_balance_constraint = Constraint(model.time_slices_through_whole_yeahr, rule=CH4Storage_Energy_balance_constraint_rule)
model.CH4Storage_level_max_in_constraint = Constraint(model.time_slices_through_whole_yeahr, rule=CH4Storage_level_max_in_constraint_rule)
model.CH4Storage_level_max_out_constraint = Constraint(model.time_slices_through_whole_yeahr, rule=CH4Storage_level_max_out_constraint_rule)
model.CH4Storage_in_or_out_or_off_constraint = Constraint(model.time_slices_through_whole_yeahr, rule=CH4Storage_in_or_out_or_off_constraint_rule)
model.connect_CH4Storage_time_slices = Constraint(model.time_slices_through_whole_yeahr, rule=connect_CH4Storage_time_slices_rule)

model.Co2Storage_level_constraint = Constraint(model.time_slices_through_whole_yeahr, rule=CO2Storage_level_constraint_rule)
model.Co2Storage_Energy_balance_constraint = Constraint(model.time_slices_through_whole_yeahr, rule=CO2Storage_Energy_balance_constraint_rule)
model.Co2Storage_level_max_out_constraint = Constraint(model.time_slices_through_whole_yeahr, rule=CO2Storage_level_max_out_constraint_rule)
model.Co2Storage_level_max_in_constraint = Constraint(model.time_slices_through_whole_yeahr, rule=CO2Storage_level_max_in_constraint_rule)
model.Co2Storage_in_or_out_or_off_constraint = Constraint(model.time_slices_through_whole_yeahr, rule=CO2Storage_in_or_out_or_off_constraint_rule)
model.connect_CO2Storage_time_slices = Constraint(model.time_slices_through_whole_yeahr, rule=connect_CO2Storage_time_slices_rule)

model.H2Storage_level_constraint = Constraint(model.time_slices_through_whole_yeahr, rule=H2Storage_level_constraint_rule)
model.H2Storage_Energy_balance_constraint = Constraint(model.time_slices_through_whole_yeahr, rule=H2Storage_Energy_balance_constraint_rule)
model.H2Storage_level_max_out_constraint = Constraint(model.time_slices_through_whole_yeahr, rule=H2Storage_level_max_out_constraint_rule)
model.H2Storage_level_max_in_constraint = Constraint(model.time_slices_through_whole_yeahr, rule=H2Storage_level_max_in_constraint_rule)
model.H2Storage_in_or_out_or_off_constraint = Constraint(model.time_slices_through_whole_yeahr, rule=H2Storage_in_or_out_or_off_constraint_rule)
model.commect_H2Storage_time_slices = Constraint(model.time_slices_through_whole_yeahr, rule=connect_H2Storage_time_slices_rule)

model.thermalStorage_level_constraint = Constraint(model.time_slices_through_whole_yeahr, rule=thermalStorage_level_constraint_rule)
model.thermalStorage_Energy_balance_constraint = Constraint(model.time_slices_through_whole_yeahr, rule=thermalStorage_Energy_balance_constraint_rule)
model.thermalStorage_level_max_out_constraint = Constraint(model.time_slices_through_whole_yeahr, rule=thermalStorage_level_max_out_constraint_rule)
model.thermalStorage_level_max_in_constraint = Constraint(model.time_slices_through_whole_yeahr, rule=thermalStorage_level_max_in_constraint_rule)
model.thermalStorage_in_or_out_or_off_constraint = Constraint(model.time_slices_through_whole_yeahr, rule=thermalStorage_in_or_out_or_off_constraint_rule)
model.connect_thermalStorage_time_slices = Constraint(model.time_slices_through_whole_yeahr, rule=connect_thermalStorage_time_slices_rule)

model.battery_level_constraint = Constraint(model.time_slices_through_whole_yeahr, rule=battery_level_constraint_rule)
model.battery_Energy_balance_constraint = Constraint(model.time_slices_through_whole_yeahr, rule=battery_Energy_balance_constraint_rule)
model.battery_level_max_out_constraint = Constraint(model.time_slices_through_whole_yeahr, rule=battery_level_max_out_constraint_rule)
model.battery_level_max_in_constraint = Constraint(model.time_slices_through_whole_yeahr, rule=battery_level_max_in_constraint_rule)
model.battery_in_or_out_or_off_constraint = Constraint(model.time_slices_through_whole_yeahr, rule=battery_in_or_out_or_off_constraint_rule)
model.connect_battery_time_slices = Constraint(model.time_slices_through_whole_yeahr, rule=connect_battery_time_slices_rule)



model.electricty_balance_constraint = Constraint(model.time_slices, rule=electricty_balance_constraint_rule)
model.heat_balance_constraint = Constraint(model.time_slices, rule=heat_balance_constraint_rule)
model.h2_balance_constraint = Constraint(model.time_slices, rule=h2_balance_constraint_rule)
model.co2_balance_contraint = Constraint(model.time_slices, rule=co2_balance_contraint_rule)
model.ch4_balance_constraint = Constraint(model.time_slices, rule=ch4_balance_constraint_rule)
model.co2_emission_constraint = Constraint(model.time_slices, rule=co2_emmission_constraint_rule)

model.PV_constraint = Constraint(model.time_slices, rule=PV_constraint_rule)
model.PEM_constraint = Constraint(model.time_slices, rule=PEM_constraint_rule)
model.PEM_minInput_constraint = Constraint(model.time_slices, rule=PEM_minInput_constraint_rule)
model.PEM_maxInput_constraint = Constraint(model.time_slices, rule=PEM_maxInput_constraint_rule)
model.DAC_constraint = Constraint(model.time_slices, rule=DAC_constraint_rule)
model.DAC_maxOuput_constraint = Constraint(model.time_slices, rule=DAC_maxOuput_constraint_rule)
model.MR_constraint = Constraint(model.time_slices, rule=MR_constraint_rule)
model.MR_CO2_constraint = Constraint(model.time_slices, rule=MR_CO2_constraint_rule)
model.MR_maxOuput_constraint = Constraint(model.time_slices, rule=MR_maxOuput_constraint_rule)
model.CHP_heat_constraint = Constraint(model.time_slices, rule=CHP_heat_constraint_rule)
model.CHP_heat_minOuput_constraint = Constraint(model.time_slices, rule=CHP_heat_minOuput_constraint_rule)
model.CHP_heat_maxOuput_constraint = Constraint(model.time_slices, rule=CHP_heat_maxOuput_constraint_rule)
model.CHP_electricity_constraint = Constraint(model.time_slices, rule=CHP_electricity_constraint_rule)
model.BO_constraint = Constraint(model.time_slices, rule=BO_constraint_rule)
model.BO_minOuput_constraint = Constraint(model.time_slices, rule=BO_minOuput_constraint_rule)
model.BO_maxOuput_constraint = Constraint(model.time_slices, rule=BO_maxOuput_constraint_rule)


## Solving the problem

In [ ]:
# declare objective
model.total_costs = Objective(expr = model.OPEX,
    sense = minimize)

In [ ]:
# Solver
solver = SolverFactory('gurobi')
#solver.options['TimeLimit'] = 3600  # Zeitlimit in Sekunden (hier 1 Stunde)
#solver.options['MIPGap'] = 0.01  # Maximale Abweichung von der optimalen Lösung in Prozent
solver.options['NodeLimit'] = 4000  # Maximale Anzahl an Knoten im Baum, die der Solver durchlaufen soll
#solver.options['IterationLimit'] = 1000  # Maximale Anzahl an Iterationen
solver.options['MIPGap'] = 0.02  # Beispiel für einen sehr kleinen MIPGap
solver.options['Heuristics'] = 0.8  # Aktiviere Heuristiken zu 80%
#solver.options['Cuts'] = 1  # Aktiviere aggressive Schnitte

# Modell lösen
results = solver.solve(model, tee=True)


In [ ]:
print(f"OPEX = {model.OPEX.value} €")

In [ ]:
import matplotlib.pyplot as plt

# Liste der Variablennamen, die geplottet werden sollen
variables_to_plot = ['P_e_buy', 'P_CH4_sell', 'P_e_PV', 'P_e_CHP', 'P_e_battery', 'P_e_PEM', 'P_e_DAC', 'P_h_BO']

# Plotten der Variablen
for var_name in variables_to_plot:
    var_object = getattr(model, var_name)
    time_steps = sorted(var_object.index_set())
    values = [var_object[t].value for t in time_steps]
    
    plt.figure()
    plt.plot(time_steps, values, marker='o')
    plt.xlabel('Time Step')
    plt.ylabel(var_name)
    plt.title(f'{var_name} over Time')
    plt.grid(True)
    plt.show()



In [ ]:
import json

# Pfad zum Speichern der JSON-Datei
file_path = 'variable_values.json'

# Sammle alle Variablenwerte und konvertiere die Schlüssel in Strings
variable_values = {}
variable_values['OPEX'] = model.OPEX.value

time_slices = model.time_slices
time_slices_through_whole_yeahr = model.time_slices_through_whole_yeahr

variables_to_save = [
    'P_e_buy', 'P_CH4_sell', 'mdot_CO2_atm', 'P_e_PEM', 'P_e_DAC', 'P_H2_MR', 'mdot_CO2_MR', 
    'P_CH4_CHP', 'P_CH4_BO', 'y_PEM', 'y_CHP', 'y_BO', 'P_e_PV', 'P_H2_PEM', 'mdot_CO2_DAC', 
    'P_CH4_MR', 'P_e_CHP', 'P_h_CHP', 'P_h_BO', 'P_e_battery', 'P_h_thermalStorage', 
    'P_H2_gasStorage', 'mdot_CO2_gasStorage', 'P_CH4_gasStorage'
]

for var_name in variables_to_save:
    variable_values[var_name] = {str(t): getattr(model, var_name)[t].value for t in time_slices}

variables_to_save_through_whole_yeahr = [
    'P_e_battery_in', 'P_e_battery_out', 'storage_level_battery', 'in_battery', 
    'out_battery', 'P_e_battery_yearly', 'P_h_thermalStorage_in', 'P_h_thermalStorage_out', 
    'storage_level_thermalStorage', 'in_thermalStorage', 'out_thermalStorage', 
    'P_h_thermalStorage_yearly', 'P_H2_gasStorage_in', 'P_H2_gasStorage_out', 
    'storage_level_H2Storage', 'in_H2Storage', 'out_H2Storage', 'P_H2_gasStorage_yearly', 
    'mdot_CO2_gasStorage_in', 'mdot_CO2_gasStorage_out', 'storage_level_CO2Storage', 
    'in_CO2Storage', 'out_CO2Storage', 'mdot_CO2_gasStorage_yearly', 'P_CH4_gasStorage_in', 
    'P_CH4_gasStorage_out', 'storage_level_CH4Storage', 'in_CH4Storage', 'out_CH4Storage', 
    'P_CH4_gasStorage_yearly'
]

for var_name in variables_to_save_through_whole_yeahr:
    variable_values[var_name] = {str(t): getattr(model, var_name)[t].value for t in time_slices_through_whole_yeahr}

with open(file_path, 'w') as f:
    json.dump(variable_values, f)

print("Variable values exported successfully.")

